In [69]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

data_train = pd.read_csv('../raw_data/application_train.csv')
data_train

data_test = pd.read_csv('../raw_data/application_test.csv')
data_test

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,456221,Cash loans,F,N,Y,0,121500.0,412560.0,17473.5,270000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
48740,456222,Cash loans,F,N,N,2,157500.0,622413.0,31909.5,495000.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
48741,456223,Cash loans,F,Y,Y,1,202500.0,315000.0,33205.5,315000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,3.0,1.0
48742,456224,Cash loans,M,N,N,0,225000.0,450000.0,25128.0,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0


In [70]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 122 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(41), object(16)
memory usage: 286.2+ MB


In [71]:
data_train.select_dtypes(include=['object']).columns

Index(['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
       'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
       'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE',
       'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE',
       'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE'],
      dtype='object')

In [72]:
data_train['TARGET'].value_counts()

0    282686
1     24825
Name: TARGET, dtype: int64

In [73]:
data_train['TARGET'].value_counts()
data_train.dtypes.value_counts()

float64    65
int64      41
object     16
dtype: int64

Recherche dans le csv les données pour le label encoder

In [74]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in data_train:
    if data_train[col].dtype == 'object':
        if len(list(data_train[col].unique())) <= 3:
            le.fit(data_train[col])
            data_train[col] = le.transform(data_train[col])
            data_test[col] = le.transform(data_test[col])

            le_count += 1
            
print('%d columns were label encoded.' % le_count)

5 columns were label encoded.


Alignement des colonnes de test et train

In [75]:
train_labels = data_train['TARGET']

# Align the training and testing data, keep only columns present in both dataframes
data_train, data_test = data_train.align(data_test, join = 'inner', axis = 1)

# Add the target back in
data_train['TARGET'] = train_labels

print('Training Features shape: ', data_train.shape)
print('Testing Features shape: ', data_test.shape)

Training Features shape:  (307511, 122)
Testing Features shape:  (48744, 121)


Affichage des correlations

In [76]:
# Find correlations with the target and sort
correlations = data_train.corr()['TARGET'].sort_values()

# Display correlations
print('Most Positive Correlations:\n', correlations.tail(15))
print('\nMost Negative Correlations:\n', correlations.head(15))

Most Positive Correlations:
 LIVE_CITY_NOT_WORK_CITY        0.032518
OWN_CAR_AGE                    0.037612
EMERGENCYSTATE_MODE            0.041955
DAYS_REGISTRATION              0.041975
FLAG_DOCUMENT_3                0.044346
REG_CITY_NOT_LIVE_CITY         0.044395
FLAG_EMP_PHONE                 0.045982
REG_CITY_NOT_WORK_CITY         0.050994
DAYS_ID_PUBLISH                0.051457
CODE_GENDER                    0.054692
DAYS_LAST_PHONE_CHANGE         0.055218
REGION_RATING_CLIENT           0.058899
REGION_RATING_CLIENT_W_CITY    0.060893
DAYS_BIRTH                     0.078239
TARGET                         1.000000
Name: TARGET, dtype: float64

Most Negative Correlations:
 EXT_SOURCE_3                 -0.178919
EXT_SOURCE_2                 -0.160472
EXT_SOURCE_1                 -0.155317
DAYS_EMPLOYED                -0.044932
FLOORSMAX_AVG                -0.044003
FLOORSMAX_MEDI               -0.043768
FLOORSMAX_MODE               -0.043226
AMT_GOODS_PRICE              -0.039645


Filtrage des colonnes pour ne récupérer que celles que l'on a besoin

In [90]:
newDf = data_train[['TARGET','SK_ID_CURR','CODE_GENDER','FLAG_OWN_CAR','CNT_CHILDREN', 'AMT_INCOME_TOTAL', 
'DAYS_BIRTH'
]]

Préparation du ML en supprimant la colonne TARGET

In [78]:
X, y = newDf.drop('TARGET', axis=1), newDf['TARGET']
X

,SK_ID_CURR,CODE_GENDER,CNT_CHILDREN,AMT_INCOME_TOTAL,DAYS_BIRTH
0,100002,1,0,202500.0,-9461
1,100003,0,0,270000.0,-16765
2,100004,1,0,67500.0,-19046
3,100006,0,0,135000.0,-19005
4,100007,1,0,121500.0,-19932
...,...,...,...,...,...
307506,456251,1,0,157500.0,-9327
307507,456252,0,0,72000.0,-20775
307508,456253,0,0,153000.0,-14966
307509,456254,0,0,171000.0,-11961


In [79]:
y.values

array([1, 0, 0, ..., 0, 1, 0], dtype=int64)

Divisez les tableaux ou les matrices en sous-ensembles d'entraînement et de test aléatoires.

In [80]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

Le randomForest est utilisé pour résoudre des problèmes de regressions et de classifications

In [81]:
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
rf.fit(X_train, y_train)

RandomForestClassifier(n_jobs=-1, random_state=42)

In [82]:
y_pred = rf.predict(X_test)

In [83]:
from sklearn.metrics import accuracy_score

score = accuracy_score(y_test, y_pred)
score

0.9166544721395704

In [84]:
conf = confusion_matrix(y_test, y_pred)
conf

array([[56351,   187],
       [ 4939,    26]], dtype=int64)

In [85]:
from sklearn.model_selection import GridSearchCV

param = { "max_features":list(range(2, 16, 1)), "max_depth":[2, 3, 4, 5, None] }
param

{'max_features': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
 'max_depth': [2, 3, 4, 5, None]}

Le gridSearch permet de calculer les valeurs optimales des hyperparamètres

In [86]:
rf = GridSearchCV(RandomForestClassifier(n_estimators=250, random_state = 42),
                    param, cv=5, n_jobs = -1)
rf

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(n_estimators=250,
                                              random_state=42),
             n_jobs=-1,
             param_grid={'max_depth': [2, 3, 4, 5, None],
                         'max_features': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                          13, 14, 15]})

In [ ]:
rf.fit(X_train, y_train)